<a target="_blank" href="https://colab.research.google.com/github/UpstageAI/cookbook/blob/main/Solar-LLM-ZeroToAll/11_tool_RAG.ipynb">
<img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a>

## Tool RAG
- Know what you know and what you don't know.


In [17]:
! pip3 install -qU  markdownify  langchain-upstage rank_bm25

In [18]:

%load_ext dotenv
%dotenv
# UPSTAGE_API_KEY

The dotenv extension is already loaded. To reload it, use:
  %reload_ext dotenv


In [19]:
import warnings

warnings.filterwarnings("ignore")

In [20]:
from langchain_upstage import ChatUpstage

llm = ChatUpstage()

In [21]:

solar_summary = """
SOLAR 10.7B: Scaling Large Language Models with Simple yet Effective Depth Up-Scaling

We introduce SOLAR 10.7B, a large language model (LLM) with 10.7 billion parameters, 
demonstrating superior performance in various natural language processing (NLP) tasks. 
Inspired by recent efforts to efficiently up-scale LLMs, 
we present a method for scaling LLMs called depth up-scaling (DUS), 
which encompasses depthwise scaling and continued pretraining.
In contrast to other LLM up-scaling methods that use mixture-of-experts, 
DUS does not require complex changes to train and inference efficiently. 
We show experimentally that DUS is simple yet effective 
in scaling up high-performance LLMs from small ones. 
Building on the DUS model, we additionally present SOLAR 10.7B-Instruct, 
a variant fine-tuned for instruction-following capabilities, 
surpassing Mixtral-8x7B-Instruct. 
SOLAR 10.7B is publicly available under the Apache 2.0 license, 
promoting broad access and application in the LLM field.
"""

In [22]:
# Tools
from langchain_core.tools import tool
import requests
import os
from tavily import TavilyClient

tavily = TavilyClient(api_key=os.environ["TAVILY_API_KEY"])


@tool
def solar_paper_search(query: str) -> str:
    """Query for research paper about solarllm, dus, llm and general AI.
    If the query is about DUS, Upstage, AI related topics, use this.
    """
    return solar_summary


@tool
def internet_search(query: str) -> str:
    """This is for query for internet search engine like Google.
    Query for general topics.
    """
    return tavily.search(query=query)


@tool
def get_news(topic: str) -> str:
    """Get latest news about a topic.
    If users are more like recent news, use this.
    """
    # https://newsapi.org/v2/everything?q=tesla&from=2024-04-01&sortBy=publishedAt&apiKey=API_KEY
    # change this to request news from a real API
    news_url = f"https://newsapi.org/v2/everything?q={topic}&apiKey={os.environ['NEWS_API_KEY']}"
    respnse = requests.get(news_url)
    return respnse.json()


tools = [solar_paper_search, internet_search, get_news]


llm_with_tools = llm.bind_tools(tools)

In [23]:
llm_with_tools.invoke("What is Solar LLM?").tool_calls


[{'name': 'solar_paper_search',
  'args': {'query': 'Solar LLM'},
  'id': 'd1dc4e7b-db66-483f-a78d-a6f00e8312c2'}]

In [24]:
llm_with_tools.invoke("What is top news about Seoul").tool_calls


[{'name': 'get_news',
  'args': {'topic': 'Seoul'},
  'id': '1194e74b-c9dd-49d1-b436-c92091f1a155'}]

In [25]:
llm_with_tools.invoke("What's best place in Seoul?").tool_calls


[{'name': 'internet_search',
  'args': {'query': 'best place in Seoul'},
  'id': '1fc63864-2f5c-4028-8bbb-e6c4e3da7e83'}]

In [26]:
def call_tool_func(tool_call):
    tool_name = tool_call["name"].lower()
    if tool_name not in globals():
        print("Tool not found", tool_name)
        return None
    selected_tool = globals()[tool_name]
    return selected_tool.invoke(tool_call["args"])

In [27]:
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser


prompt_template = PromptTemplate.from_template(
    """
    Please provide answer for question from the following context. 
    ---
    Question: {question}
    ---
    Context: {context}
    """
)
chain = prompt_template | llm | StrOutputParser()

In [28]:
# Smart RAG, Self-Improving RAG
import os
from tavily import TavilyClient


def tool_rag(question):
    for _ in range(3): # try 3 times
        tool_calls = llm_with_tools.invoke(question).tool_calls
        if tool_calls:
            break
        else:
            print("try again")

    if not tool_calls:
        return "I'm sorry, I don't have an answer for that."
    
    print(tool_calls)
    context = ""
    for tool_call in tool_calls:
        context += str(call_tool_func(tool_call))

    chain = prompt_template | llm | StrOutputParser()
    return chain.invoke({"context": context, "question": question})

In [29]:
tool_rag("What is Solar llm?")

[{'name': 'solar_paper_search', 'args': {'query': 'Solar llm'}, 'id': 'aba35656-834f-4f6b-ae30-7f2c35a66686'}]


'Solar 10.7B is a large language model (LLM) with 10.7 billion parameters, developed using a method called depth up-scaling (DUS) which includes depthwise scaling and continued pretraining. It is designed to perform various natural language processing (NLP) tasks and has been shown to outperform other LLMs, including Mixtral-8x7B-Instruct. Solar 10.7B is publicly available under the Apache 2.0 license, which allows for broad access and application in the LLM field.'

In [30]:
tool_rag("What is news about Tesla?")

[{'name': 'get_news', 'args': {'topic': 'Tesla'}, 'id': '3f97738c-6ac7-4a0d-9bf1-3e3f3e622dae'}]


KeyError: 'NEWS_API_KEY'

In [ ]:
tool_rag("iPhone 13 spec?")

[{'name': 'internet_search', 'args': {'query': 'iPhone 13 spec'}, 'id': '7b89b621-fd4b-4bfe-9eec-dac21354d93c'}]


'The iPhone 13 specs include a 6.1-inch display with a 2532 x 1170 pixel resolution and a 60Hz refresh rate. It is equipped with the Apple A15 Bionic chipset, 4GB of RAM, and 128GB of storage that is not expandable. The device has a 12MP dual camera and a 12MP front camera. The iPhone 13 has a ceramic shield glass and a weight of 6.14 ounces. It has a built-in stereo speaker and microphone, as well as a Lightning connector. The battery life is up to 19 hours for video playback and up to 75 hours for audio playback. The device also supports various languages and has built-in accessibility features.'

# Excercise
Solar LLM is small, so it might not give the best results for complex tasks. For those, you can use larger LLMs like GPT-4. However, for quick answers and summaries, using a small size LLM like Solar can give better performance and efficiency.

Please note that good engineering involves making things work with limited components.
![Engineering](figures/engineering.jpg)